In [114]:
import csv
import sys
import os
import pandas as pd
import numpy as np
import random

In [115]:
def get_random_element(l):
    num = len(l)
    index = random.randint(0, num-1)
    return l[index]

In [116]:
def generate_job_arrive_time(interval, job_num):
    arrive_interval = np.random.poisson(interval, job_num)
    arrive_time_list = np.cumsum(arrive_interval).astype(float)
    arrive_time_list = [x - arrive_time_list[0] for x in arrive_time_list]
    return arrive_time_list
    

In [117]:
def fill_comp_comm(worker_num, row, index, total, tp_comp_time, tp_comm_size):
    if worker_num == 1:
        row["comp_time"] = random.randint(1, 10)
        row["comm_size"] = 1
    else:
        if index % 2 == 0:
            row["comp_time"] = tp_comp_time
            row["comm_size"] = tp_comm_size
        elif index % 2 == 1:
            row["comp_time"] = 0
            row["comm_size"] = 1
        if index == total - 1: # embedding traffic
            row["comp_time"] = 0
            row["comm_size"] = random.randint(2, 4)
    

In [118]:
total_worker_num = 0

In [119]:

def generate_one_trace(template_trace_dir, generate_trace_dir, job_id, arrive_time):
    worker_num = get_random_element([1,2,4,8,2,4,8])
    global total_worker_num
    total_worker_num += worker_num
    iter_num = random.randint(20, 30)
    # if worker_num == 1:
    #     iter_num *= 5
    with open(os.path.join(template_trace_dir, f'{worker_num}-worker.csv'), 'r') as f:
        reader = csv.DictReader(f)
        fieldnames = reader.fieldnames
        modified_rows = []
        rows = list(reader)  # 将所有行存入列表以便识别最后一个元素
        total_rows = len(rows)
        tp_comp_time = random.randint(1, 10)
        tp_comm_size = random.randint(6, 10)
        
        for index, row in enumerate(rows):
            row["arrive_time"] = arrive_time
            row["job_id"] = job_id
            row["iter_num"] = iter_num
            row["worker_num"] = worker_num

            fill_comp_comm(worker_num, row, index, total_rows, tp_comp_time, tp_comm_size)
            
            row["upstream"] = "\"" + row["upstream"] + "\""
            row["downstream"] = "\"" + row["downstream"] + "\""
            modified_rows.append(row)
    with open(os.path.join(generate_trace_dir, f'ddl-job-{job_id}.csv'), 'w') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for rows in modified_rows:
            for v in dict(rows).values():
                f.write(str(v) + ",")
            f.write("\n")

In [120]:
def generate_traces(template_trace_dir,generate_trace_dir, job_num, arrive_time_list):
    for i in range(job_num):
        arrive_time = arrive_time_list[i]
        generate_one_trace(template_trace_dir, generate_trace_dir, i, arrive_time)

In [121]:
template_trace_dir = "job-template"
generate_trace_dir = "job-generate"
job_num = 10
# 3ms comes a job
interval = 10
arrive_time_list = generate_job_arrive_time(interval, job_num)
print(arrive_time_list)
generate_traces(template_trace_dir,generate_trace_dir, job_num, arrive_time_list)
print(total_worker_num)

[np.float64(0.0), np.float64(11.0), np.float64(18.0), np.float64(32.0), np.float64(44.0), np.float64(52.0), np.float64(55.0), np.float64(68.0), np.float64(76.0), np.float64(84.0)]
39


In [122]:
import numpy as np
iter_time_list = {
    "ideal": np.array([4,10,3.75]),
    "crux":  np.array([4.18,37.8,3.87]),
    "crux+": np.array([4.19,33.6,3.96]),
    "worst": np.array([17.9,10.5,4.21]),
    "equal": np.array([7.18, 14.6, 4.12]),
    "JFP":   np.array([7.95,15.7,3.9])
}
res_dict = {}
for method, times in iter_time_list.items():
    if method != "ideal":
        res_dict[method] = np.sum(times / iter_time_list["ideal"])
for method, times in res_dict.items():
    print(f"{method}: {times}")
    res_dict[method] /= res_dict["JFP"]

import matplotlib.pyplot as plt
# plt.bar(res_dict.keys(), res_dict.values())

crux: 5.856999999999999
crux+: 5.463500000000001
worst: 6.647666666666666
equal: 4.353666666666666
JFP: 4.5975
